In [1]:
import numba
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path # creating dir

In [2]:
adata = ...

# Clustering

In [3]:
adata.X = adata.layers["logcounts"]
obs_key = "Harmony"
sc.pp.neighbors(adata, use_rep=obs_key)

2024-10-22 22:57:39.883010: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-22 22:57:40.361137: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-22 22:57:40.542075: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-22 22:57:41.487896: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-22 22:57:59.298051: W tensorflow/comp

In [4]:
resolutions = [0.1, 0.3, 0.5, 0.7, 1, 1.5, 2, 3]

for resolution in resolutions:
    key_added = f"leiden_{resolution}"
    sc.tl.leiden(adata, resolution=resolution, key_added=key_added, flavor='igraph', n_iterations=-1)

In [ ]:
leiden_groups = ["leiden_1", "leiden_0.5", "leiden_0.3", "leiden_0.1", "leiden_0.7", "leiden_1.5", "leiden_2", "leiden_3"]

dir_name = 'Anno-All'

adata.X = adata.layers['logcounts']
for group in leiden_groups:
    key_added = f"dea_{group}"
    key_added_filtered = f"{key_added}_filtered"

    sc.tl.rank_genes_groups(
        adata, groupby=group, method="wilcoxon", key_added=key_added
    )
    sc.tl.filter_rank_genes_groups(
        adata,
        key=key_added,
        key_added=key_added_filtered,
    )
    df = sc.get.rank_genes_groups_df(adata, group=None, 
                                     key=key_added_filtered, 
                                     pval_cutoff=0.05)
    df = df.dropna(subset=['names'])
    
    Path(dir_name).mkdir(parents=True, exist_ok=True)
    df.to_excel(f'{dir_name}/{key_added_filtered}.xlsx', index=False)